In [30]:
"""Steps:
    1) Load and merge data (Customers + Transactions)
    2) Feature engineering (total spend, average order, tenure, region)
    3) Outlier capping + optional log(1+x) transform
    4) Try multiple clustering algorithms with different parameters:
        - K-Means
        - Agglomerative Clustering
        - Gaussian Mixture
        - DBSCAN
    5) Record metrics (DB, silhouette, etc.) and pick the best DB.
    6) Output final labels and top configurations.
    7) Print top 3 lookalikes for the first 20 customers and save to Lookalike.csv
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

# Load and preprocess the dataset
data = pd.read_csv('Merged_DataFrame.csv', dayfirst=True)

# Ensure proper date formatting and drop rows with missing essential information
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'], errors='coerce', dayfirst=True)
data.dropna(subset=['CustomerID', 'ProductID', 'TransactionDate'], inplace=True)

# Sort the dataset by transaction date for a time-based split
data_sorted = data.sort_values(by='TransactionDate').reset_index(drop=True)

n = len(data_sorted)
print(f"Total transactions in dataset: {n}")

# Determine the split point for a 65/35 time-based split
train_ratio = 0.65
split_index = int(n * train_ratio)
split_date = data_sorted.loc[split_index, 'TransactionDate']

print(f"Split date (65th percentile): {split_date}")

# Create training and test datasets based on the split date
train_data = data[data['TransactionDate'] < split_date]
test_data = data[data['TransactionDate'] >= split_date]

print("Train set size:", len(train_data))
print("Test set size:", len(test_data))

# Fill missing values in key columns of the training data
train_data.loc[:, 'Region'] = train_data['Region'].fillna('Unknown')
train_data.loc[:, 'Category'] = train_data['Category'].fillna('Unknown')
train_data.loc[:, 'ProductName'] = train_data['ProductName'].fillna('Unknown')
train_data.loc[:, 'TotalValue'] = train_data['TotalValue'].fillna(0)
train_data.loc[:, 'Quantity'] = train_data['Quantity'].fillna(0)

# Aggregate customer-level data for analysis and feature creation
user_agg = train_data.groupby('CustomerID').agg(
    TotalExpenses=('TotalValue', 'sum'),
    TransactionCount=('TransactionDate', 'count'),
    LastPurchaseDate=('TransactionDate', 'max'),
    Region=('Region', lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')  # Handle empty mode
).reset_index()

# Compute additional metrics like average order value (AOV) and recency
user_agg['AOV'] = user_agg['TotalExpenses'] / user_agg['TransactionCount']
reference_date_train = train_data['TransactionDate'].max()
user_agg['Recency'] = (reference_date_train - user_agg['LastPurchaseDate']).dt.days

# Measure diversity in categories purchased
user_agg['CategoryDiversity'] = train_data.groupby('CustomerID')['Category'].nunique().values

# Analyze how each customer interacts with product categories
user_cat_matrix = train_data.pivot_table(
    index='CustomerID',
    columns='Category',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)
user_cat_distribution = user_cat_matrix.div(user_cat_matrix.sum(axis=1), axis=0).fillna(0)

# Create text-based profiles for customers using product and category information
def concat_product_text(df):
    return " ".join(df['ProductName'] + " " + df['Category'])

user_text_df = (
    train_data.groupby('CustomerID')[['ProductName', 'Category']]
    .apply(concat_product_text)
    .reset_index(name='AllProductsText')
)

# Use TF-IDF to process text-based profiles and limit features for simplicity
tfidf = TfidfVectorizer(stop_words='english', max_features=100)
tfidf_matrix = tfidf.fit_transform(user_text_df['AllProductsText'])
user_text_df = user_text_df.set_index('CustomerID')
tfidf_user_ids = user_text_df.index.tolist()
tfidf_index_map = {uid: i for i, uid in enumerate(tfidf_user_ids)}

# Combine all features (aggregates, category distribution, and text profiles)
user_agg = user_agg.set_index('CustomerID')
combined_features_df = user_agg.join(user_cat_distribution, how='left').join(user_text_df, how='left')
combined_features_df.fillna(0, inplace=True)

# Encode categorical data (Region) and standardize numeric features
region_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
region_encoded = region_encoder.fit_transform(combined_features_df[['Region']])

numeric_cols = ['TotalExpenses', 'AOV', 'Recency', 'CategoryDiversity'] + list(user_cat_distribution.columns)
numeric_data = combined_features_df[numeric_cols].values
scaler = StandardScaler()
numeric_data_scaled = scaler.fit_transform(numeric_data)

# Build context vectors for each user by combining features
final_user_ids = combined_features_df.index.tolist()
context_vectors = []
for i, uid in enumerate(final_user_ids):
    numeric_region_vec = np.hstack([numeric_data_scaled[i], region_encoded[i]])
    text_vec = tfidf_matrix[tfidf_index_map[uid]].toarray().flatten() if uid in tfidf_index_map else np.zeros(tfidf_matrix.shape[1])
    combined_vec = np.hstack([numeric_region_vec, text_vec])
    context_vectors.append(combined_vec)

context_vectors = np.array(context_vectors)
context_user_similarity = cosine_similarity(context_vectors)
user_index_map = {uid: idx for idx, uid in enumerate(final_user_ids)}

# Perform collaborative filtering using SVD on the user-item interaction matrix
user_item_matrix = train_data.pivot_table(
    index='CustomerID',
    columns='ProductID',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)

svd = TruncatedSVD(n_components=20, random_state=42)
user_embeddings_svd = svd.fit_transform(user_item_matrix)
cf_user_ids = user_item_matrix.index.tolist()
cf_user_index_map = {uid: i for i, uid in enumerate(cf_user_ids)}
transaction_sim = cosine_similarity(user_embeddings_svd)

# Combine similarities from both content and collaborative filtering approaches
def get_combined_user_similarity(user_id, alpha=0.5):
    if (user_id not in user_index_map) or (user_id not in cf_user_index_map):
        return None
    ctx_idx = user_index_map[user_id]
    cf_idx = cf_user_index_map[user_id]
    sim_context_vec = context_user_similarity[ctx_idx]
    sim_cf_vec = transaction_sim[cf_idx]
    return alpha * sim_context_vec + (1 - alpha) * sim_cf_vec

def get_top_lookalikes(user_id, top_n=3, alpha=0.5):
    combined_sim = get_combined_user_similarity(user_id, alpha=alpha)
    if combined_sim is None:
        return []
    pairs = [(uid, combined_sim[i]) for i, uid in enumerate(final_user_ids) if uid != user_id]
    return sorted(pairs, key=lambda x: x[1], reverse=True)[:top_n]

# Evaluate the model by measuring category overlap between users and their lookalikes
def evaluate_lookalike_hit_rate_by_category(test_data, lookalike_dict):
    user_test_categories = test_data.groupby('CustomerID')['Category'].apply(set).to_dict()
    hit_rates = []
    for user_id, lookalikes in lookalike_dict.items():
        if user_id not in user_test_categories or not user_test_categories[user_id]:
            continue
        user_cats = user_test_categories[user_id]
        union_lk_cats = set()
        for lk_id, _ in lookalikes:
            if lk_id in user_test_categories:
                union_lk_cats |= user_test_categories[lk_id]
        overlap = user_cats & union_lk_cats
        hit_rate = len(overlap) / len(user_cats)
        hit_rates.append(hit_rate)
    return np.mean(hit_rates) if hit_rates else 0.0

# Generate recommendations and evaluate
lookalike_dict = {uid: get_top_lookalikes(uid, top_n=3, alpha=0.5) for uid in test_data['CustomerID'].unique()}
avg_hit_rate_cat = evaluate_lookalike_hit_rate_by_category(test_data, lookalike_dict)
print(f"Average Category Overlap Hit Rate on Test Data: {avg_hit_rate_cat:.3f}")

# =============================================================
# 6. PRINT TOP 3 LOOKALIKE USERS FOR FIRST 20 CUSTOMERS AND SAVE TO CSV
# =============================================================

# Define the first 20 CustomerIDs (C0001 - C0020)
# Adjust the range if CustomerIDs don't start at C0001 or have different formatting
first_20_customer_ids = [f"C{str(i).zfill(4)}" for i in range(1, 20)]

# Ensure these CustomerIDs exist in the lookalike_dict
existing_customers = [cid for cid in first_20_customer_ids if cid in lookalike_dict]

print("\n=== Top 3 Lookalike Users for the First 20 Customers ===\n")
lookalike_output = []

for cid in first_20_customer_ids:
    lookalikes = lookalike_dict.get(cid, [])
    if not lookalikes:
        print(f"CustomerID: {cid} has no lookalikes.")
        lookalike_output.append({
            'CustomerID': cid,
            'Lookalike1': None,
            'Score1': None,
            'Lookalike2': None,
            'Score2': None,
            'Lookalike3': None,
            'Score3': None
        })
        continue
    print(f"CustomerID: {cid}")
    temp_dict = {'CustomerID': cid}
    for rank in range(1, 4):
        if rank <= len(lookalikes):
            lk_id, score = lookalikes[rank - 1]
            print(f"  Lookalike {rank}: {lk_id} with similarity score {score:.4f}")
            temp_dict[f'Lookalike{rank}'] = lk_id
            temp_dict[f'Score{rank}'] = round(score, 4)
        else:
            print(f"  Lookalike {rank}: None")
            temp_dict[f'Lookalike{rank}'] = None
            temp_dict[f'Score{rank}'] = None
    lookalike_output.append(temp_dict)
    print()  # For better readability

# Convert the lookalike_output list of dictionaries to a DataFrame
lookalike_df = pd.DataFrame(lookalike_output)

# Ensure that all 20 customers are represented in the DataFrame
# If some customers were missing in lookalike_dict, they are already handled above

# Save to Lookalike.csv
lookalike_df.to_csv('Lookalike.csv', index=False)
print("Lookalike mappings for the first 20 customers have been saved to 'Lookalike.csv'")


Total transactions in dataset: 1000
Split date (65th percentile): 2024-08-15 15:59:00
Train set size: 650
Test set size: 350
Average Category Overlap Hit Rate on Test Data: 0.684

=== Top 3 Lookalike Users for the First 20 Customers ===

CustomerID: C0001
  Lookalike 1: C0156 with similarity score 0.7596
  Lookalike 2: C0129 with similarity score 0.6839
  Lookalike 3: C0020 with similarity score 0.5633

CustomerID: C0002
  Lookalike 1: C0030 with similarity score 0.5552
  Lookalike 2: C0007 with similarity score 0.5518
  Lookalike 3: C0138 with similarity score 0.5490

CustomerID: C0003
  Lookalike 1: C0134 with similarity score 0.8185
  Lookalike 2: C0181 with similarity score 0.6975
  Lookalike 3: C0144 with similarity score 0.5021

CustomerID: C0004
  Lookalike 1: C0025 with similarity score 0.5472
  Lookalike 2: C0124 with similarity score 0.5438
  Lookalike 3: C0065 with similarity score 0.5233

CustomerID: C0005
  Lookalike 1: C0096 with similarity score 0.5025
  Lookalike 2: C00